# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. This is the dataset to use for this assignment. Note: The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Ann Arbor, Michigan, United States**, and the stations the data comes from are shown on the map below.

In [2]:
import matplotlib.pyplot as plt
%matplotlib notebook
import mplleaflet
import pandas as pd
import numpy as np
from matplotlib.backends.backend_agg import FigureCanvasAgg
from matplotlib.figure import Figure

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')

FileNotFoundError: File b'data/C2A2_data/BinSize_d400.csv' does not exist

In [3]:
def generate_plot():
    df = pd.read_csv('data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')
    
    for x in range(len(df['Date'].values)):
        if df['Date'][x][5:] == '02-29':
            df.drop(x,inplace=True)

    df['Year'] = df['Date'].apply(lambda x: x[:4])
    df['MD'] = df['Date'].apply(lambda x: x[5:])
    df15 = df[df['Year'] == '2015']
    df15.reset_index(inplace=True)
    df15.drop('index',inplace=True,axis=1)
    df = df[df['Year'] != '2015']
    df.sort_values('Date',inplace=True)
    df.reset_index(inplace=True)
    df.rename(columns={'index':'Old_Index'},inplace=True)
    #df['Date'] = pd.to_datetime(df['Date'])

    dfmin = df[['MD','Data_Value']].groupby('MD').min().reset_index()
    dfmax = df[['MD','Data_Value']].groupby('MD').max().reset_index()
    
    df15min = df15[['Date','Data_Value','MD']].groupby('MD').min()
    df15min.reset_index(inplace=True)
    df15max = df15[['Date','Data_Value','MD']].groupby('MD').max()
    df15max.reset_index(inplace=True)
    
    brokemax = df15max[dfmax['Data_Value'] < df15max['Data_Value']]
    brokemax.reset_index(inplace=True)
    brokemin = df15min[dfmin['Data_Value'] > df15min['Data_Value']]
    brokemin.reset_index(inplace=True)
    
    plt.figure()
    plt.plot(range(365),dfmax['Data_Value'],markersize=1,linewidth=1,color='green',label='Temp Max (2005-2014)')
    plt.plot(range(365),dfmin['Data_Value'],markersize=1,linewidth=1,color='blue',label='Temp Min (2005-2014)')
    plt.gca().fill_between(range(365),dfmax['Data_Value'],dfmin['Data_Value'], facecolor='grey',alpha=0.35)
    plt.scatter(brokemax['index'],brokemax['Data_Value'],c='black',s=5,label='Broken Max Record (2015)')
    plt.scatter(brokemin['index'],brokemin['Data_Value'],c='purple',s=5,label='Broken Min Record (2015)')
    plt.legend(frameon=False)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)
    plt.ylabel('Temperature (Tenths of a degree, C)')
    plt.title('Max and Min Temperatures in Ann Arbor, MI')
    plt.xticks(np.linspace(0,365,12), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
    for item in plt.gca().xaxis.get_ticklabels():
        item.set_rotation(45)
    #plt.yticks(np.linspace(-40,40,9))
    
    #return plt.gcf()

In [4]:
generate_plot()

FileNotFoundError: File b'data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv' does not exist

In [2]:
#fig = Figure()
#canvas = FigureCanvasAgg(fig)
#canvas.print_png('James Monia - Assignment2.png')